In [1]:
import requests
import json
import pandas as pd
import os
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 環境変数からAPIキーを取得
app_id = os.getenv('RECIPE_APP_ID')

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=' + app_id)

json_data = json.loads(res.text)

json_data

{'result': {'large': [{'categoryId': '30',
    'categoryName': '人気メニュー',
    'categoryUrl': 'https://recipe.rakuten.co.jp/category/30/?rafcid=wsc_r_cl_1081839724132064301'},
   {'categoryId': '31',
    'categoryName': '定番の肉料理',
    'categoryUrl': 'https://recipe.rakuten.co.jp/category/31/?rafcid=wsc_r_cl_1081839724132064301'},
   {'categoryId': '32',
    'categoryName': '定番の魚料理',
    'categoryUrl': 'https://recipe.rakuten.co.jp/category/32/?rafcid=wsc_r_cl_1081839724132064301'},
   {'categoryId': '33',
    'categoryName': '卵料理',
    'categoryUrl': 'https://recipe.rakuten.co.jp/category/33/?rafcid=wsc_r_cl_1081839724132064301'},
   {'categoryId': '14',
    'categoryName': 'ご飯もの',
    'categoryUrl': 'https://recipe.rakuten.co.jp/category/14/?rafcid=wsc_r_cl_1081839724132064301'},
   {'categoryId': '15',
    'categoryName': 'パスタ',
    'categoryUrl': 'https://recipe.rakuten.co.jp/category/15/?rafcid=wsc_r_cl_1081839724132064301'},
   {'categoryId': '16',
    'categoryName': '麺・粉物料理',
    '

In [3]:
parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    cat_list = [category['categoryId'], "","",category['categoryId'],category['categoryName']]
    df.loc[len(df)] = cat_list


# 中カテゴリ
for category in json_data['result']['medium']:
    cat_list = [
        category['parentCategoryId'],
        category['categoryId'],
        "",
        str(category['parentCategoryId'])+"-"+str(category['categoryId']),
        category['categoryName']
        ]
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']
    df.loc[len(df)] = cat_list
    

# 小カテゴリ
for category in json_data['result']['small']:
    cat_list = [
        parent_dict[category['parentCategoryId']],
        category['parentCategoryId'],
        category['categoryId'],
        parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),
        category['categoryName']
    ]
    df.loc[len(df)] = cat_list
df

,category1,category2,category3,categoryId,categoryName
0,30,,,30,人気メニュー
1,31,,,31,定番の肉料理
2,32,,,32,定番の魚料理
3,33,,,33,卵料理
4,14,,,14,ご飯もの
...,...,...,...,...,...
2156,31,720,2130,31-720-2130,ローストポーク
2157,33,721,2131,33-721-2131,味付け卵
2158,18,722,2183,18-722-2183,キャロットラペ
2159,23,723,2182,23-723-2182,ミルフィーユ鍋


In [4]:
recipe_list = []

# jsonファイルを開く
with open('recipe_list.json', 'r') as f:
  # jsonファイルを読み込む
  recipe_list = json.load(f)
print(len(recipe_list))

3384


In [5]:
for catid in df['categoryId']:
  print(f'{catid}=====================================')
  recipe_dict = {}
  
  load_count = 0
  json_data = None
  while True:
    ranking_url = f'https://app.rakuten.co.jp/services/api/Recipe/CategoryRanking/20170426?applicationId={app_id}&categoryId={catid}'
  
    # レシピデータのランキングを取得
    res = requests.get(ranking_url)
    status_code = res.status_code
    
    if status_code == 200:
      json_data = json.loads(res.text)
      break
    
    # 一定時間スリープ
    wait_time = 2 ** load_count
    print(f'wait {wait_time} sec')
    time.sleep(wait_time)
    
    load_count += 1
  
  # result がの項目があるか確認
  if json_data.get('result') is None:
    continue
  
  if len(json_data['result']) == 0:
    continue
  
  for result in json_data['result']:
    recipe_id = result['recipeId']
    print(f'{recipe_id}---------------------------------')
    
    # recipe_list に　recipe_id が含まれている場合はスキップ
    flag = False
    for recipe in recipe_list:
      if recipe['recipeId'] == recipe_id:
        flag = True
        break
    if flag:
      print(f'{recipe_id} : skip')
      continue
    
    recipe_dict['recipeId'] = recipe_id
    recipe_dict['recipeTitle'] = result['recipeTitle']
    recipe_dict['recipeUrl'] = result['recipeUrl']

    # print(f'Title : {result["recipeTitle"]}')

    # 取得したレシピ url からレシピデータを取得
    recipe_url = result['recipeUrl']
    res = requests.get(recipe_url)

    # レシピデータを取得するため html をパース

    soup = BeautifulSoup(res.text, 'html.parser')

    # body を取得
    body = soup.find('body')

    # recipe_material__item が割り当てられている要素を取得
    recipe_material__items = body.find_all(class_='recipe_material__item')

    material_item_list = []

    for matrial_item in recipe_material__items:
      material_name = matrial_item.find(class_='recipe_material__item_name').text
      material_serving = matrial_item.find(class_='recipe_material__item_serving').text

      # 改行をなくす
      material_name = material_name.replace('\n','')
      material_serving = material_serving.replace('\n','')

      # print(f'{material_name} : {material_serving}')

      material_item_list.append({
          'material_name': material_name,
          'material_serving': material_serving
      })
  
    recipe_dict['recipe_material'] = material_item_list

    # 作り方の項目を取得 (recipe_howto)
    recipe_howto = body.find(class_='recipe_howto')

    # recipe_howto__text が割り当てられている要素を取得
    recipe_howto_texts = recipe_howto.find_all(class_='recipe_howto__text')
    
    if recipe_howto_texts is None:
      print(f'{recipe_id} は作り方がありません')
      continue

    howto_list = []
    for index, howto in enumerate(recipe_howto_texts):
      howto_list.append(howto.text)
      # print(f'{index+1} : {howto.text}') 

    recipe_dict['recipe_howto'] = howto_list
    
    recipe_list.append(recipe_dict)
    
    print(f"取得完了 : {result['recipeTitle']}")



30=====================================
1290001623---------------------------------
1290001623 : skip
1570003792---------------------------------
1570003792 : skip
1010011230---------------------------------
1010011230 : skip
1180006148---------------------------------
1180006148 : skip
31=====================================
wait 1 sec
1400014946---------------------------------
1400014946 : skip
1670008437---------------------------------
1670008437 : skip
1720001297---------------------------------
1720001297 : skip
1270000186---------------------------------
1270000186 : skip
32=====================================
wait 1 sec
1650018044---------------------------------
1650018044 : skip
1790010634---------------------------------
1790010634 : skip
1440002551---------------------------------
1440002551 : skip
1280023259---------------------------------
1280023259 : skip
33=====================================
wait 1 sec
1310001611---------------------------------
1310001611 : skip
1

In [6]:
print(len(recipe_list))

5089


In [7]:
# 重複したレシピを削除
tmp_recipe_list = []
print(len(recipe_list))
for recipe in recipe_list:
  
  # tmp_recipe_list に recipe が含まれているか確認
  include_flag = False
  for tmp_recipe in tmp_recipe_list:
    
    # 含まれている場合はスキップ
    if recipe['recipeId'] == tmp_recipe['recipeId']:
      include_flag = True
      break
    
  # 含まれていない場合は tmp_recipe_list に追加
  if not include_flag:
    tmp_recipe_list.append(recipe)
recipe_list = tmp_recipe_list
      
print(len(tmp_recipe_list))

# recipe_list を json に変換
json_data = json.dumps(recipe_list, ensure_ascii=False, indent=4)
# json ファイルに保存
with open('recipe_list.json', 'w') as f:
  f.write(json_data)

5089
4351


In [8]:
prompt_base = """
材料
{materials}

作り方
{howto}
"""

receipe_dataset = []

for recipe in recipe_list:
  title = recipe['recipeTitle']
  print(title)
  
  material_list = recipe['recipe_material']
  matelias = ""
  for m in material_list:
    matelias += f"{m['material_name']} : {m['material_serving']}\n"
  
  howto_list = recipe['recipe_howto']
  howto = ""
  for index, h in enumerate(howto_list):
    howto += f"{index+1} : {h}\n"
  
  # 作り方を表示
  recipe_str = prompt_base.format(materials=matelias, howto=howto)
  
  receipe_prompt = {
    'instruction': "料理の作り方を教えてください",
    'input' : title,
    'output' : recipe_str
  }
  receipe_dataset.append(receipe_prompt)

# json ファイルに保存
with open('llm_datasets/recipe_dataset.json', 'w') as f:
  json.dump(receipe_dataset, f, ensure_ascii=False, indent=2)

ささみときゅうりの中華風酢の物
我が家の人気者！！鶏の唐揚げ
ふっくらやわらか★茹で鯖
プロ直伝！ニラともやしの卵炒め
♪♪吉●家より美味しい？ツユだく吉野家風牛丼♪♪
牛乳で簡単♪明太子クリームパスタ
どっかのお店の新製品やつ！　うどーなっつ！
超簡単ホテルの味！じゃがいもの冷製ヴィシソワーズ
【失敗なしっ！】白菜と豚肉のミルフィーユ鍋♪
セブンの完全再現！たことブロッコリーのバジルサラダ
グルテンフリー！米粉のバナナパウンドケーキ
簡単100％ジュースで作るゼリーレシピ
5分でできちゃう!じゃがいもとベーコンの簡単炒め物
かつおの漬け
めちゃめちゃゴハンがすすむ！茄子と豚肉の味噌炒め
スイカ入り❤オリゴ糖で❤おうちカフェ寒天デザート♪
＊そうめんに＊ピリ辛中華風つけ汁
ほぼ三○矢サイダー☆
夕顔と油揚げのフワフワ煮物
ポリ袋で簡単！調味料1つで☆瓜の漬け物
シンプルだけど箸が止まらない☆ししとうの焼き浸し♪
5分で速攻！もやしだけdeとん平焼き☆
tiktokで話題沸騰中！【やみつきたまご】
昔ながらの「焼きめし」
ポテチ越えの美味しさ！！山芋のり塩バタ〜
レンジで簡単★カボチャのチーズ焼き
さっぱり！簡単！無限ナスのポン酢で焼き浸し。
本格！麻婆豆腐 ☆　美味しい　☆
意外と簡単！フライパンでチーズタッカルビ
子供ウケ抜群♫ツナマヨコーンピザ
簡単！タコとブロッコリーとジャガイモのバジルソース
パンが足りなくなる！きのことベーコンのアヒージョ
簡単！炊飯器でカオマンガイ！海南鶏飯！
失敗なし！超簡単！豚肉とゴーヤの中華炒め
野菜たっぷり！！鮭のちゃんちゃん焼き
ジャガイモとチーズだけ！！【簡単ガレット】
♡臭みもなくプリプリに♪簡単海老の下ごしらえ♡
簡単！失敗なしのローストビーフ♡絶品ソース！
生ハムとじゃがいもだけ！大人のおつまみポテトサラダ
昔ながらのカスタードプリン❤
ご飯がすすむ♪ 豚肉となすの味噌炒め☆
ミキサーで超簡単☆濃厚かぼちゃプリン
材料はココアとはちみつだけ!?濃厚生チョコ!?♪
♪♪絶品★煮込みハンバーグ～きのこソース～♪♪
☆時短☆豚こまとピーマンのチンジャオロース
冷めてもおいしい! 鶏もも肉のマヨポン和え
離乳食後期～手づかみ食べに！豆腐ハンバーグ
ズッキーニと玉ねぎとベーコンの炒め物
☆我が家の子供用チャーハン☆
麺がくっつかな